In [1]:
import json

import pandas as pd
from db import *

/home/jovyan/work/db.py:8: SAWarning: Did not recognize type 'geometry' of column 'geom'
  mapping_hotel = Table(


In [2]:
reports_room_inventory = Table("reports_room_inventory", metadata, autoload_with=engine)

In [ ]:
from datetime import date, timedelta

# Define the start and end dates
start_date = date(2023, 10, 1)
end_date = date(2023, 10, 31)

# Create a list of dates between the start and end dates
date_list = [
    start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)
]

# Create a DataFrame with the date column
df = pd.DataFrame({"date": date_list})
df["room_id"] = 3179
df["quantity"] = 5
df["basis"] = "allocation"
df["charter"] = False
df["allocation_group_id"] = 5
df["top_up"] = 0
df["stop_sale_protected"] = False
df["protected_rooms"] = 0

from sqlalchemy.dialects.postgresql import insert

stmt = insert(reports_room_inventory).values(df.to_dict(orient="records"))

with Session(engine) as session:
    session.execute(stmt)
    session.commit()

In [ ]:
from sqlalchemy import update

stmt = (
    update(reports_room_inventory)
    .where(reports_room_inventory.c.room_id == 3183)
    .values(quantity=0)
)

with Session(engine) as session:
    session.execute(stmt)
    session.commit()

In [ ]:
fanar_topup = pd.read_excel("fanar_topup.xlsx", parse_dates=["date"])
fanar_topup.shape

In [ ]:
salalah_topup = pd.read_excel("salalah_topup.xlsx", parse_dates=["date"])
salalah_topup.shape

In [ ]:
salalah_classic = 3381
fanar_deluxe = 3180

stmt = (
    update(reports_room_inventory)
    .where(
        reports_room_inventory.c.room_id == 3180,
        reports_room_inventory.c.allocation_group_id
        == 5,
    )
    .values(basis="allocation")
)

with Session(engine) as session:
    session.execute(stmt)
    session.commit()

In [3]:
topup_data = pd.read_excel("topup.xlsx", parse_dates=["date"])
topup_data.head()

,date,topup,room_id,allocation_group_id
0,2023-12-01,5,3381,1
1,2023-12-03,5,3381,1
2,2023-12-04,5,3381,1
3,2023-12-05,5,3381,1
4,2023-12-06,5,3381,1


In [4]:
from sqlalchemy import update

# add rooms to existing top up
print("date", "\t", "topup", "\t", "result", "\t", "new_result")
for row in topup_data.to_dict(orient="records"):
    select_room = select(reports_room_inventory.c.top_up).where(
        reports_room_inventory.c.room_id == row["room_id"],
        reports_room_inventory.c.date == row["date"],
        reports_room_inventory.c.allocation_group_id == row["allocation_group_id"],
    )

    with Session(engine) as session:
        result = session.scalar(select_room)

        stmt = (
            update(reports_room_inventory)
            .where(
                reports_room_inventory.c.room_id == row["room_id"],
                reports_room_inventory.c.date == row["date"],
                reports_room_inventory.c.allocation_group_id
                == row["allocation_group_id"],
            )
            .values(top_up=row["topup"] + result)
        )
        session.execute(stmt)
        session.commit()
        new_result = session.scalar(select_room)

        print(row["date"], "\t", row["topup"], "\t", result, "\t", new_result)

date 	 topup 	 result 	 new_result
2023-12-01 00:00:00 	 5 	 0 	 5
2023-12-03 00:00:00 	 5 	 0 	 5
2023-12-04 00:00:00 	 5 	 0 	 5
2023-12-05 00:00:00 	 5 	 3 	 8
2023-12-06 00:00:00 	 5 	 3 	 8
2023-12-07 00:00:00 	 5 	 3 	 8
2023-12-08 00:00:00 	 5 	 3 	 8
2023-12-09 00:00:00 	 5 	 3 	 8
2023-12-10 00:00:00 	 5 	 0 	 5
2023-12-11 00:00:00 	 5 	 0 	 5
2023-12-12 00:00:00 	 5 	 0 	 5
2023-12-24 00:00:00 	 5 	 0 	 5
2023-12-25 00:00:00 	 5 	 0 	 5
2023-12-28 00:00:00 	 5 	 0 	 5
2023-12-31 00:00:00 	 5 	 0 	 5
2024-01-01 00:00:00 	 5 	 0 	 5
2024-01-02 00:00:00 	 5 	 0 	 5
2024-01-03 00:00:00 	 5 	 0 	 5
2024-01-04 00:00:00 	 5 	 0 	 5
2024-01-05 00:00:00 	 5 	 0 	 5
2024-01-06 00:00:00 	 5 	 0 	 5
2024-01-07 00:00:00 	 5 	 0 	 5
2024-01-08 00:00:00 	 5 	 0 	 5
2024-01-09 00:00:00 	 5 	 0 	 5
2024-01-10 00:00:00 	 5 	 0 	 5
2024-01-11 00:00:00 	 5 	 2 	 7
2024-01-12 00:00:00 	 5 	 2 	 7
2024-01-13 00:00:00 	 5 	 2 	 7
2024-01-14 00:00:00 	 5 	 2 	 7
2024-01-15 00:00:00 	 5 	 0 	 5
2024-

In [ ]:
allocation_update = pd.read_excel("allocation_data.xlsx", parse_dates=["date"])
allocation_update.head()

In [ ]:
for row in allocation_update.to_dict(orient="records"):
    select_room = select(reports_room_inventory.c.quantity).where(
        reports_room_inventory.c.room_id == row["room_id"],
        reports_room_inventory.c.date == row["date"],
        reports_room_inventory.c.allocation_group_id == row["allocation_group_id"],
    )

    with Session(engine) as session:
        result = session.scalar(select_room)

        stmt = (
            update(reports_room_inventory)
            .where(
                reports_room_inventory.c.room_id == row["room_id"],
                reports_room_inventory.c.date == row["date"],
                reports_room_inventory.c.allocation_group_id
                == row["allocation_group_id"],
            )
            .values(quantity=row["quantity"] + result)
        )
        session.execute(stmt)
        session.commit()
        new_result = session.scalar(select_room)

        print(row["date"], "\t", row["quantity"], "\t", result, "\t", new_result)